In [2]:
import tensorflow as tf

# Allow GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU available.")

# Initialize GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

1 Physical GPUs, 1 Logical GPUs
Num GPUs Available:  1


In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Rutas a las carpetas de entrenamiento, prueba y validación para imágenes y máscaras
train_frames_dir = '../Frames/TRAIN'
train_Landmarks_dir = './Landmarks/TRAIN'
test_frames_dir = '../Frames/TEST'
test_Landmarks_dir = './Landmarks/TEST'
val_frames_dir = '../Frames/VAL'
val_Landmarks_dir = './Landmarks/VAL'

# Funciones para cargar y preprocesar imagen y máscara y convertirlo a solo un canal
def load_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image / 255.0
    return image

def load_landmarks(file_path):
    images = []
    for filename in os.listdir(file_path):
        image=cv2.imread(os.path.join(file_path, filename))
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(grayscale_image)

    stacked_image = np.stack(images, axis=-1)  # axis=-1 indica la dimensión de los canales

    return stacked_image

train_image_files = [os.path.join(train_frames_dir, filename) for filename in os.listdir(train_frames_dir)]
train_Landmarks_files = [os.path.join(train_Landmarks_dir, filename) for filename in os.listdir(train_Landmarks_dir)]
test_image_files = [os.path.join(test_frames_dir, filename) for filename in os.listdir(test_frames_dir)]
test_Landmarks_files = [os.path.join(test_Landmarks_dir, filename) for filename in os.listdir(test_Landmarks_dir)]
val_image_files = [os.path.join(val_frames_dir, filename) for filename in os.listdir(val_frames_dir)]
val_Landmarks_files = [os.path.join(val_Landmarks_dir, filename) for filename in os.listdir(val_Landmarks_dir)]

train_images = [load_image(file) for file in train_image_files]
train_Landmarks = [load_landmarks(file) for file in train_Landmarks_files]

test_images = [load_image(file) for file in test_image_files]
test_Landmarks = [load_landmarks(file) for file in test_Landmarks_files]

val_images = [load_image(file) for file in val_image_files]
val_Landmarks = [load_landmarks(file) for file in val_Landmarks_files]

train_images = np.array(train_images)
train_Landmarks = np.array(train_Landmarks)
test_images = np.array(test_images)
test_Landmarks = np.array(test_Landmarks)
val_images = np.array(val_images)
val_Landmarks = np.array(val_Landmarks)

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, BatchNormalization, Activation, ReLU, Conv2DTranspose

# Llamamos a la funcion del modelo U-Net desde un archivo externo dentro de la ruta R:\Codes\Reto\Modelos
from Modelos.model3 import unet_model

# Crear el modelo U-Net
model = unet_model(input_shape=(112, 112, 1), num_classes=7, output_activation='softmax')

ImportError: attempted relative import with no known parent package

In [5]:
BATCH_SIZE = 16

# with tf.device('CPU'):
#     train_dataset = tf.data.Dataset.from_tensor_slices((train_images[:700], train_Landmarks[:700]))
#     train_dataset = train_dataset.batch(BATCH_SIZE)

#     val_dataset = tf.data.Dataset.from_tensor_slices((val_images[:300], val_Landmarks[:300]))
#     val_dataset = val_dataset.batch(BATCH_SIZE)

#     test_dataset = tf.data.Dataset.from_tensor_slices((test_images[:300], test_Landmarks[:300]))
#     test_dataset = test_dataset.batch(BATCH_SIZE)

with tf.device('CPU'):
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_Landmarks))
    train_dataset = train_dataset.batch(BATCH_SIZE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_Landmarks))
    val_dataset = val_dataset.batch(BATCH_SIZE)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_images[:2550], test_Landmarks[:2550]))
    test_dataset = test_dataset.batch(BATCH_SIZE)

In [6]:
import tensorflow as tf

def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    y_true_flat = tf.keras.backend.flatten(y_true)
    y_pred_flat = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_flat * y_pred_flat)
    return (2.0 * intersection + smooth) / (tf.keras.backend.sum(y_true_flat) + tf.keras.backend.sum(y_pred_flat) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

# model.compile(optimizer=tf.keras.optimizers.Adam(), loss=dice_loss, metrics=[dice_coefficient, 'accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])

In [7]:
# Agregamos un callback para guardar el modelo cada 5 épocas y otro para detener el entrenamiento si no hay mejora en 10 épocas
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, mode='min', monitor='val_loss')
]

# Entrenamos el modelo
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10
933/933 [==============================] - 149s 146ms/step - loss: 5.0999 - accuracy: 0.1392 - val_loss: 5.0444 - val_accuracy: 0.0279
Epoch 2/10
933/933 [==============================] - 130s 140ms/step - loss: 5.0300 - accuracy: 0.1858 - val_loss: 5.0204 - val_accuracy: 0.0126
Epoch 3/10
933/933 [==============================] - 133s 143ms/step - loss: 5.1112 - accuracy: 0.1509 - val_loss: 5.1833 - val_accuracy: 7.9719e-05
Epoch 4/10
933/933 [==============================] - 129s 139ms/step - loss: 5.1834 - accuracy: 0.0151 - val_loss: 5.1833 - val_accuracy: 7.9719e-05
Epoch 5/10
933/933 [==============================] - 132s 142ms/step - loss: 5.1833 - accuracy: 0.0730 - val_loss: 5.1833 - val_accuracy: 7.9719e-05
Epoch 6/10
933/933 [==============================] - 133s 142ms/step - loss: 5.1833 - accuracy: 0.1555 - val_loss: 5.1833 - val_accuracy: 7.9719e-05
Epoch 7/10
933/933 [==============================] - 132s 142ms/step - loss: 5.1833 - accuracy: 0.1373 - va

In [ ]:
evaluation = model.evaluate(test_dataset)

In [8]:

# Assuming you have a single input image or a batch of images
input_image = test_images[0:1]

# Perform prediction
predictions = model.predict(input_image)

# Get the predicted class for each pixel
predicted_classes = np.argmax(predictions)

# Get the ground truth class for each pixel
ground_truth = np.argmax(test_Landmarks[0:1])

# Plot the ground truth
import matplotlib.pyplot as plt
plt.imshow(ground_truth[0], cmap='jet')
plt.show()

# Plot the prediction
plt.imshow(predicted_classes[0], cmap='jet')
plt.show()

# Plot the input image
plt.imshow(input_image[0], cmap='jet')
plt.show()

1/1 [==============================] - 1s 751ms/step


IndexError: invalid index to scalar variable.

In [ ]:
# Save model history information to a json file
import json
import time
import matplotlib.pyplot as plt

# Generate a folder name
name_folder = 'mascara_{}'.format(time.strftime("%Y%m%d-%H%M%S"))
path = '../../Modelos/Pruebas/{}/'.format(name_folder)
os.mkdir(path)

# Save history to a json file
path_json = '{}history.json'.format(path)

with open(path_json, 'w') as fp:
    json.dump(history.history, fp)

# Save model
path_model = '{}model.h5'.format(path)
model.save(path_model)

# Save model summary to a txt file
path_summary = '{}summary.txt'.format(path)
with open(path_summary, 'w') as fp:
    model.summary(print_fn=lambda x: fp.write(x + '\n'))

# Save model metrics to a txt file
path_metrics = '{}metrics.txt'.format(path)
with open(path_metrics, 'w') as fp:
    fp.write('Test loss: {}\n'.format(evaluation[0]))
    fp.write('Test dice coefficient: {}\n'.format(evaluation[1]))
    fp.write('Test accuracy: {}\n'.format(evaluation[2]))

# Generate a folder inside the folder of the model for the images
path_images = '{}images/'.format(path)
os.mkdir(path_images)

# Generate a folder inside the folder of the images for plot images
path_plot_images = '{}plot_images/'.format(path_images)
os.mkdir(path_plot_images)

# Generate a folder inside the folder of the images for prediction images
path_test_images = '{}predicitons/'.format(path_images)
os.mkdir(path_test_images)

# Save prediction images
# Función para visualizar una muestra de imágenes y sus máscaras de segmentación predichas
def visualize_segmentation(images, masks, predictions):
    num_samples = len(images)

    for i in range(num_samples):
        plt.figure(figsize=(12, 6))

        # Imagen original
        plt.subplot(1, 3, 1)
        plt.imshow(images[i])
        plt.title('Imagen Original')

        # Máscara de segmentación real
        plt.subplot(1, 3, 2)
        plt.imshow(masks[i], cmap='gray')
        plt.title('Máscara Real')

        # Máscara de segmentación predicha
        plt.subplot(1, 3, 3)
        plt.imshow(predictions[i], cmap='gray')
        plt.title('Máscara Predicha')

        plt.tight_layout()
        plt.savefig('{}{}.png'.format(path_test_images, i))

# Supongamos que tienes un conjunto de datos de prueba con imágenes y máscaras
# Puedes utilizar el modelo para obtener las máscaras predichas en el conjunto de prueba
predictions = model.predict(test_dataset)

# Elije algunas muestras aleatorias para visualizar
num_samples_to_visualize = 5
sample_indices = np.random.choice(len(test_images[:10]), num_samples_to_visualize, replace=False)

sample_images = [test_images[i] for i in sample_indices]
sample_masks = [test_Landmarks[i] for i in sample_indices]
sample_predictions = [predictions[i] for i in sample_indices]

# Llama a la función para visualizar las imágenes y máscaras
visualize_segmentation(sample_images, sample_masks, sample_predictions)

# Save plot images
for i in ['accuracy', 'loss']:
    plt.figure()
    plt.plot(history.history[i])
    plt.plot(history.history['val_{}'.format(i)])
    plt.title('Model {}'.format(i))
    plt.ylabel(i)
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig('{}{}.png'.format(path_plot_images, i))

# 